In [6]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO


In [2]:
stat_ad = pd.read_csv('stat_ad_set_product.csv')
ad_set = pd.read_csv('ad_set.csv')

In [4]:
stat_ad.head()

,id,ad_set_product_id,ad_set_id,pid,uid,ad_date,cnt_impression,cnt_click,cnt_unique_impression,cnt_unique_click,cnt_transaction,cnt_faved_user,cnt_purchase,paid_point,free_point,modified_at
0,117055172,220315,80505,127851054,11559144,2021-03-30,6284,28,2911,28,0,0,0,0,0,2021-03-30 23:54:1617148454.SSS
1,117055174,183767,65058,29458942,1568593,2021-03-30,4251,22,937,22,0,0,0,0,0,2021-03-31 00:02:1617148971.SSS
2,117055176,211885,76111,55080422,3070670,2021-03-30,6424,31,1500,31,0,0,0,0,0,2021-03-31 00:02:1617148975.SSS
3,117055180,217123,79050,148754756,4673541,2021-03-30,1271,2,632,2,0,0,0,0,0,2021-03-30 19:01:1617130869.SSS
4,117055202,219053,79942,139154737,7078901,2021-03-30,582,19,307,18,0,0,0,0,0,2021-03-30 23:42:1617147723.SSS


In [5]:
ad_set.head()

,date,ad_set_product_id,ad_set_id,keyword,bid_price
0,2021-04-02,233579,83450,원피스,1000
1,2021-04-03,233579,83450,원피스,1000
2,2021-04-02,233581,83450,귀걸이,410
3,2021-04-03,233581,83450,귀걸이,410
4,2021-04-02,233935,83423,탈모방지샴푸,100


In [8]:
q = '''
SELECT YEAR||MONTH||DAY AS date, ad_set_product_id, bid_price
FROM bun_log_db.api_event_type_impression_ad
WHERE YEAR||MONTH||DAY BETWEEN '20210330' AND '20210403'
GROUP BY 1, 2, 3
'''
api = pd.read_sql(q, con=bun_dw)

In [9]:
api.head()

,date,ad_set_product_id,bid_price
0,20210330,219830,50
1,20210330,220500,50
2,20210330,219852,50
3,20210330,217544,50
4,20210330,219662,50


In [10]:
api.bid_price = api.bid_price.astype(int)
max_bid = api.groupby(['date', 'ad_set_product_id'], as_index=False)['bid_price'].max()

In [11]:
max_bid

,date,ad_set_product_id,bid_price
0,20210330,100172,100
1,20210330,103818,50
2,20210330,104130,50
3,20210330,105154,80
4,20210330,105236,50
...,...,...,...
49654,20210403,95355,50
49655,20210403,96250,55
49656,20210403,98176,50
49657,20210403,98470,50


In [13]:
p_max_bid = api.groupby(['ad_set_product_id', 'date'], as_index=False)['bid_price'].max()

In [15]:
p_max_bid.sort_values(['ad_set_product_id', 'date'])

,ad_set_product_id,date,bid_price
0,100169,20210331,50
1,100172,20210330,100
2,1002,20210401,50
3,100216,20210402,100
4,100471,20210331,605
...,...,...,...
49654,99030,20210401,600
49655,99183,20210401,50
49656,9931,20210401,50
49657,9985,20210401,50


In [17]:
cnt = p_max_bid.groupby('ad_set_product_id', as_index=False)['date'].count()

In [19]:
p_max_bid[p_max_bid['ad_set_product_id'].isin(cnt[cnt['date'] >= 2]['ad_set_product_id'].unique())].sort_values(['ad_set_product_id', 'date'])

,ad_set_product_id,date,bid_price
62,107816,20210402,50
63,107816,20210403,50
123,112965,20210330,50
124,112965,20210331,50
125,112965,20210402,50
...,...,...,...
49593,92463,20210403,50
49618,95355,20210330,50
49619,95355,20210403,50
49652,99029,20210330,55


In [30]:
march31 = p_max_bid[p_max_bid['date'] == '20210331']
april01 = p_max_bid[p_max_bid['date'] == '20210401']

In [23]:
compare = pd.merge(march, april, on='ad_set_product_id')
compare.columns = ['ad_set_product_id', 'ad_date1', 'max_bid_price1', 'ad_date2', 'max_bid_price2']

In [26]:
compare.head()

,ad_set_product_id,ad_date1,max_bid_price1,ad_date2,max_bid_price2
0,133715,20210331,150,20210401,150
1,134585,20210331,55,20210401,90
2,139733,20210331,50,20210401,50
3,143726,20210331,50,20210401,50
4,147310,20210331,50,20210401,50


In [25]:
compare.describe()

,max_bid_price1,max_bid_price2
count,6237.000000,6237.000000
mean,215.000000,188.168992
std,250.293734,238.631858
min,50.000000,50.000000
25%,50.000000,50.000000
50%,80.000000,60.000000
75%,275.000000,200.000000
max,1000.000000,1000.000000


In [32]:
p_max_bid[p_max_bid['date'] == '20210330'].describe()

,bid_price
count,9637.000000
mean,169.494656
std,221.536412
min,50.000000
25%,50.000000
50%,55.000000
75%,155.000000
max,1000.000000


In [33]:
p_max_bid[p_max_bid['date'] == '20210331'].describe()

,bid_price
count,10021.000000
mean,177.221335
std,228.247424
min,50.000000
25%,50.000000
50%,55.000000
75%,195.000000
max,1000.000000


In [34]:
p_max_bid[p_max_bid['date'] == '20210401'].describe()

,bid_price
count,10182.000000
mean,166.781575
std,222.819589
min,50.000000
25%,50.000000
50%,55.000000
75%,150.000000
max,1000.000000


In [35]:
p_max_bid[p_max_bid['date'] == '20210402'].describe()

,bid_price
count,10044.000000
mean,167.950518
std,220.716043
min,50.000000
25%,50.000000
50%,60.000000
75%,150.000000
max,1000.000000


In [36]:
p_max_bid[p_max_bid['date'] == '20210403'].describe()

,bid_price
count,9775.000000
mean,165.509463
std,218.151113
min,50.000000
25%,50.000000
50%,55.000000
75%,150.000000
max,1000.000000


In [41]:
p_max_bid.groupby('date')['ad_set_product_id'].nunique()

date
20210330     9637
20210331    10021
20210401    10182
20210402    10044
20210403     9775
Name: ad_set_product_id, dtype: int64

In [89]:
q = '''
SELECT bid_price, ref_source, count(*) as cnt
FROM bun_log_db.api_event_type_impression_ad
WHERE YEAR||MONTH||DAY = '20210330'
GROUP BY 1, 2
'''
m30 = pd.read_sql(q, con=bun_dw)

In [90]:
q = '''
SELECT bid_price, ref_source, count(*) as cnt
FROM bun_log_db.api_event_type_impression_ad
WHERE YEAR||MONTH||DAY = '20210331'
GROUP BY 1, 2
'''
m31 = pd.read_sql(q, con=bun_dw)

In [91]:
q = '''
SELECT bid_price, ref_source, count(*) as cnt
FROM bun_log_db.api_event_type_impression_ad
WHERE YEAR||MONTH||DAY = '20210401'
GROUP BY 1, 2
'''
a1 = pd.read_sql(q, con=bun_dw)

In [92]:
q = '''
SELECT bid_price, ref_source, count(*) as cnt
FROM bun_log_db.api_event_type_impression_ad
WHERE YEAR||MONTH||DAY = '20210402'
GROUP BY 1, 2
'''
a2 = pd.read_sql(q, con=bun_dw)

In [93]:
m30.groupby('ref_source')['cnt'].sum()

ref_source
ca    2500080
sa    3914591
Name: cnt, dtype: int64

In [112]:
m30.groupby('ref_source')['bid_price'].sum()

ref_source
ca       50
sa    84450
Name: bid_price, dtype: int64

In [94]:
m31.groupby('ref_source')['cnt'].sum()

ref_source
ca    2473389
sa    3821529
Name: cnt, dtype: int64

In [111]:
m31.groupby('ref_source')['bid_price'].sum()

ref_source
ca    71340
sa    81905
Name: bid_price, dtype: int64

In [95]:
a1.groupby('ref_source')['cnt'].sum()

ref_source
ca    2085506
sa    3146080
Name: cnt, dtype: int64

In [110]:
a1.groupby('ref_source')['bid_price'].sum()

ref_source
ca    76880
sa    84550
Name: bid_price, dtype: int64

In [96]:
a2.groupby('ref_source')['cnt'].sum()

ref_source
ca    2005855
sa    2889078
Name: cnt, dtype: int64

In [113]:
a2.groupby('ref_source')['bid_price'].sum()

ref_source
ca    80360
sa    80990
Name: bid_price, dtype: int64

In [99]:
q = '''
SELECT year||month||day as date, count(distinct ad_set_product_id) as product_cnt
FROM bun_log_db.api_event_type_impression_ad
WHERE YEAR||MONTH||DAY between '20210330' and '20210402'
GROUP BY 1
'''
p_cnt = pd.read_sql(q, con=bun_dw)

In [101]:
p_cnt.sort_values('date')

,date,product_cnt
1,20210330,9637
0,20210331,10021
2,20210401,10182
3,20210402,10044


In [68]:
x = pd.merge(m30, m31, on='bid_price', how='outer', suffixes=('_330', '_331'))

In [69]:
x = pd.merge(x, a1, on='bid_price' , how='outer')
x.columns = ['bid_price', '330', '331', '401']

In [70]:
x = pd.merge(x, a2, on='bid_price' , how='outer')
x.columns = ['bid_price', '330', '331', '401', '402']

In [71]:
x.bid_price = x.bid_price.astype(int)
x.sort_values('bid_price', ascending=False).to_csv('x.csv', index=False)

In [72]:
x.head()

,bid_price,330,331,401,402
0,395,358.0,3479.0,608.0,937.0
1,360,3756.0,6531.0,7154.0,3073.0
2,640,259.0,136.0,214.0,186.0
3,100,141963.0,148799.0,197136.0,194661.0
4,270,6138.0,7400.0,6303.0,7667.0


In [73]:
x.sum()

bid_price      90950.0
330          6414671.0
331          6294918.0
401          5231586.0
402          4894933.0
dtype: float64

In [83]:
x[x['331'] > x['401']].sort_values('bid_price', ascending=False)

,bid_price,330,331,401,402
176,995,NaN,838.0,20.0,2.0
172,990,18.0,28.0,1.0,NaN
20,795,152.0,104.0,66.0,71.0
124,790,62.0,5.0,1.0,94.0
8,785,3195.0,6201.0,4265.0,3636.0
136,775,2178.0,3080.0,2067.0,2121.0
46,770,475.0,449.0,244.0,291.0
107,745,10071.0,19194.0,18277.0,14596.0
104,740,444.0,71.0,64.0,NaN
80,735,253.0,96.0,30.0,57.0


In [84]:
x[x['331'] < x['401']].sort_values('bid_price', ascending=False)

,bid_price,330,331,401,402
33,1000,3904.0,14080.0,21170.0,11817.0
175,965,NaN,2.0,558.0,769.0
27,950,2031.0,10725.0,27972.0,72.0
73,910,601.0,2562.0,12764.0,4970.0
92,900,645.0,2586.0,5468.0,195.0
...,...,...,...,...,...
156,75,30857.0,38892.0,48379.0,30903.0
19,70,56550.0,78951.0,89016.0,84692.0
91,65,44084.0,64607.0,105686.0,105424.0
133,60,144269.0,196830.0,220468.0,193210.0


In [86]:
x[x['331'] > x['401']]['bid_price'].describe()

count     42.000000
mean     525.714286
std      228.701455
min       50.000000
25%      348.750000
50%      545.000000
75%      690.000000
max      995.000000
Name: bid_price, dtype: float64

In [87]:
x[x['331'] < x['401']]['bid_price'].describe()

count     128.000000
mean      460.468750
std       259.712468
min        55.000000
25%       238.750000
50%       440.000000
75%       681.250000
max      1000.000000
Name: bid_price, dtype: float64

In [77]:
x[x['331'] > x['401']].sum()

bid_price      22080.0
330          4999729.0
331          4478935.0
401          2510765.0
402          2469250.0
dtype: float64

58940

In [78]:
x[x['331'] < x['401']].sum()

bid_price      58940.0
330          1413546.0
331          1815980.0
401          2717709.0
402          2417411.0
dtype: float64